<a href="https://colab.research.google.com/github/gyeonh/machine_learning/blob/main/12_custom_models_and_training_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#12.1 텐서플로 훑어보기

텐서플로(Tensorflow)
* 하나의 라이브러리 그 이상임
* 광범위한 라이브러리 생태계를 가짐

In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version은 코랩 명령입니다.
    %tensorflow_version 2.x
except Exception:
    pass

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
tf.random.set_seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

#12.2 넘파이처럼 텐서플로 사용하기

텐서플로 API는 텐서를 순환시킨다.

<br>

텐서
* 한 연산에서 다른 연산으로 흐른다.
* 넘파이 ndarray와 매우 비슷하다.
* 일반적으로 다차원 배열이다.
* 스칼라 값도 가질 수 있다.

## 12.2.1 텐서와 연산

tf.constant() 함수로 텐서 생성


In [2]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # 행렬을 텐서로 

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

텐서의 크기와 데이터 타입 확인하기

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

인덱스 참조하기(넘파이와 매우 비슷하게 작동함)

In [6]:
t[:, 1:]    # 모든 행, 0열 제외 모든 열

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [18]:
t[..., 2, tf.newaxis]   # 모든 행, 1열

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[3.],
       [6.]], dtype=float32)>

 텐서플로는 모든 종류의 텐서 연산이 가능하다

In [8]:
t + 10      # 모든 원소에 10 더하기

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)    # 모든 원소를 제곱

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t)     # @ = tf.matmul() : 행렬의 곱 연산

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

케라스 API는 keras.backend에 자체적인 저수준 API를 가지고 있다. 

tf.keras에서 exp(), sqrt() 같은 함수들은 보통 상응하는 텐서플로 연산을 호출하는 것이 전부이다. 다른 케라스 구현에 적용할 수 있는 코드를 짜고 싶다면 케라스 함수를 사용해야한다. 하지만 케라스 저수준 함수는 텐서플로에서 제공하는 함수의 일부만 지원하기 때문에 <br>이 책에서는 텐서플로 연산을 직접 사용하도록 한다.

In [11]:
# keras.backend를 사용하는 간단한 예
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

##12.2.2 텐서와 넘파이

텐서와 넘파이는 함께 사용하기 좋다. 넘파이 배열로 텐서를 만들거나 넘파이 연산을 텐서에 적용할 수 있고, 그 반대도 가능하다.

In [12]:
a = np.array([2., 4., 5.])   # 넘파이 배열
tf. constant(a)     # 넘파이 배열을 텐서로

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [13]:
t.numpy()   # 텐서를 넘파이 배열로

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [14]:
tf.square(a)    # 텐서 연산을 넘파이 배열에 적용

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [15]:
np.square(t)    # 넘파이 연산을 텐서에 적용

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

##12.2.3 타입 변환